CIFAR-10  
Using dataset from kaggle for image classification into 10 classes.

In [10]:
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets, models
import matplotlib.pyplot as plt

In [11]:
transform_train=transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


In [ ]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

classes = {'plane', 'bird', 'car', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'}

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #2 convolutional layers and 2 fully connected layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))   # [batch, 32, 16, 16]
        x = self.pool(F.relu(self.conv2(x)))   # [batch, 64, 8, 8]
        x = x.view(-1, 64 * 8 * 8)             # flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)                        # no softmax, handled by loss
        return x
model=CNN()



Loss and Optimizer

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Training Loop

In [16]:
epochs = 10
best_acc = 0.0
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs=model(images)
        loss= criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(trainloader):.4f}')

Epoch [1/10], Loss: 0.0029
Epoch [1/10], Loss: 0.0059
Epoch [1/10], Loss: 0.0088
Epoch [1/10], Loss: 0.0118
Epoch [1/10], Loss: 0.0147
Epoch [1/10], Loss: 0.0176
Epoch [1/10], Loss: 0.0206
Epoch [1/10], Loss: 0.0235
Epoch [1/10], Loss: 0.0264
Epoch [1/10], Loss: 0.0293
Epoch [1/10], Loss: 0.0322
Epoch [1/10], Loss: 0.0350
Epoch [1/10], Loss: 0.0378
Epoch [1/10], Loss: 0.0407
Epoch [1/10], Loss: 0.0435
Epoch [1/10], Loss: 0.0463
Epoch [1/10], Loss: 0.0491
Epoch [1/10], Loss: 0.0518
Epoch [1/10], Loss: 0.0545
Epoch [1/10], Loss: 0.0572
Epoch [1/10], Loss: 0.0599
Epoch [1/10], Loss: 0.0627
Epoch [1/10], Loss: 0.0654
Epoch [1/10], Loss: 0.0680
Epoch [1/10], Loss: 0.0705
Epoch [1/10], Loss: 0.0732
Epoch [1/10], Loss: 0.0759
Epoch [1/10], Loss: 0.0786
Epoch [1/10], Loss: 0.0813
Epoch [1/10], Loss: 0.0838
Epoch [1/10], Loss: 0.0865
Epoch [1/10], Loss: 0.0892
Epoch [1/10], Loss: 0.0918
Epoch [1/10], Loss: 0.0945
Epoch [1/10], Loss: 0.0972
Epoch [1/10], Loss: 0.0998
Epoch [1/10], Loss: 0.1025
E

Validation

In [17]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

acc = 100 * correct / total
print(f'Accuracy of the model on the test images: {acc} %')
if acc > best_acc:
    best_acc = acc
    torch.save(model.state_dict(), 'best_model.pth')
    print("Model saved!")

Accuracy of the model on the test images: 73.94 %
Model saved!


# BetterCNN
Now try to achieve better model accuracy